In [342]:
""" Playgame routine.  Plays one game between StratA and StratB.  Outputs return to A
    Added output of cards and plays to allow ML"""

def playgame(GameDeck, StratA, StratB, verbose = False):

    # Deal 
    cardA = GameDeck.deal()
    cardB = GameDeck.deal()
    if verbose: print("Card A: ", cardA, " Card B: ", cardB)
    playA = ""
    playB = ""
    
    # Player A decides
    playA = StratA.play(cardA,"A")
    if verbose: print("Player A: ", playA)

    # if Player A pass, showdown for $2    
    if playA == "Pass":
        if cardA > cardB:
            payout = 1
        elif cardB > cardA:
            payout = -1
        else:
            payout = 0
    # if Player A raises, player B decides
    else:
        playB = StratB.play(cardB,"B")
        if verbose: print("Player B: ", playB)
        
        #if player B calls, showdown for $4
        if playB == "Call":
            if cardA > cardB:
                payout = 5
            elif cardB > cardA:
                payout = -5
            else:
                assert (cardA == cardB)
                payout = 0
        # if player B folds, A gets the ante
        else:
            payout = 1
    if verbose: 
        print("Payout: ",payout)
        print("")
    return {'winA':payout,'playA':playA,'playB':playB,'cardA':cardA,'cardB':cardB}
        

In [343]:
""" Deck class  Defines the deck.  For now only a discrete set 0 - n-1"""

class Deck:
    def __init__(self, decksize):
        self.decksize = decksize
        self.cards = range(self.decksize)      
        
    def deal(self):
        import random              # is it ok to have this here?
        card_delt = random.randint(0,self.decksize -1 )
        return card_delt
    

In [344]:
"""  Strategy Class.  Sets standards for all strategies 
    Create a subclass for each strategy"""

class Strategy:
    def __init__(self):
        pass
        # self.gamedeck = GameDeck
        # self.decksize = GameDeck.decksize
        
    def play(self,mycard,player):
        """ determine strategy for player, having been dealt card mycard.  
        If player = "A" return either 'Pass' or 'Raise' 
        If player = 'B' return either 'Fold' or 'Call' """ 
        pass
    



In [345]:
""" Vector based strategy.  

    Paramaterized by a vector giving probability of aggressive (Raise/Call) strategy for each card.
    
    Old strategies
    
    Random - [1/2,1/2,1/2,1/2,1/2,1/2]
    Simple - e.g [0,0,0,1,1,1] 
    bluff - e.g [p,p,p,1,1,1]
    optimal - A [2/3,0,0,0,1,1]  B [0, 1/3,1/3,1,1,1]"""

class vectorstrat(Strategy):
    
    def __init__(self,aggprobs):
        self.aggprobs = aggprobs
        
    def play(self,mycard,player):
        import random
        if random.random() > self.aggprobs[mycard]:
            if player == "A":
                return 'Pass'
            else:
                return 'Fold'
        else:
            if player == "A":
                return "Raise"
            else:
                return 'Call'

        

In [346]:
"""Challenge Routine.
Plays n games between two strategies and returns the net result.
Used for testing
"""

def challenge(num_games,strata,stratb,strataname = "",stratbname = "",verbose = False):

    decksize = 6
    d = Deck(decksize)

    a_net_wins = 0

    if verbose: print("Player A: ", strataname,"   Player B: ", stratbname )

    for i in range (num_games):
        a_net_wins += playgame(d,strata,stratb)['winA']
        if verbose and i % 1000 == 0: print(i," games played")

    if verbose: print(strataname, " won $", a_net_wins, "  $", a_net_wins / num_games, " per game.")
    
    return a_net_wins

In [40]:
"""Tournament.  Runs a tournament among several A and B strategies.

    Used some in testing, but not important in v1"""

num_games = 10000
verbose = True

Astrategies = []
s = vectorstrat([.5,.5,.5,.5,.5,.5])
Astrategies += [{'strat':s,'name':"Random    "}]
s = vectorstrat([0,1,1,1,1,1])
Astrategies += [{'strat':s,'name':"Simple 1  "}]
s = vectorstrat([0,0,1,1,1,1])
Astrategies += [{'strat':s,'name':"Simple 2  "}]
s = vectorstrat([0,0,0,1,1,1])
Astrategies += [{'strat':s,'name':"Simple 3  "}]
s = vectorstrat([0.6666,0,0,0,1,1])
Astrategies += [{'strat':s,'name':"Optimal   "}]

Bstrategies = []
s = vectorstrat([.5,.5,.5,.5,.5,.5])
Bstrategies += [{'strat':s,'name':"Random    "}]
s = vectorstrat([0,1,1,1,1,1])
Bstrategies += [{'strat':s,'name':"Simple 1"}]
s = vectorstrat([0,0,1,1,1,1])
Bstrategies += [{'strat':s,'name':"Simple 2"}]
s = vectorstrat([0,0,0,1,1,1])
Bstrategies += [{'strat':s,'name':"Simple 3"}]
s = vectorstrat([0,0.333,0.33333,1,1,1])
Bstrategies += [{'strat':s,'name':"Optimal   "}]

for bstrat in Bstrategies:
    bstrat['profit'] = 0
    
for astrat in Astrategies:
    astrat['profit'] = 0
    for bstrat in Bstrategies:
        if verbose: print (astrat['name']," playing ",bstrat['name'])
        awins = challenge(num_games,astrat['strat'],bstrat['strat'],verbose = False)
        astrat['profit'] += awins
        bstrat['profit'] -= awins
        astrat[bstrat['name']] = awins
    astrat['average'] = astrat['profit'] / (num_games * len(Bstrategies))
    
for bstrat in Bstrategies:
    bstrat['average'] = bstrat['profit'] / (num_games * len(Astrategies))
    

print()
print ("Tournament Results -- winnings for Strategy A")
print("                        Strategy B")
print ("Strategy A    | ", end = "")
for b in Bstrategies:
    print(b['name'], end =" | ")
print("Total")
print("_______________________________________________")
for a in Astrategies:
    print(a['name'], end = "    |")
    for b in Bstrategies:
        print(a[b['name']], end = "         |")
    print(a['profit'])
print("Total         |", end = "")
for b in Bstrategies:
    print(-b['profit'], end = "         |")
print(-sum([b['profit'] for b in Bstrategies]))
    
        
    
                            


Random      playing  Random    
Random      playing  Simple 1
Random      playing  Simple 2
Random      playing  Simple 3
Random      playing  Optimal   
Simple 1    playing  Random    
Simple 1    playing  Simple 1
Simple 1    playing  Simple 2
Simple 1    playing  Simple 3
Simple 1    playing  Optimal   
Simple 2    playing  Random    
Simple 2    playing  Simple 1
Simple 2    playing  Simple 2
Simple 2    playing  Simple 3
Simple 2    playing  Optimal   
Simple 3    playing  Random    
Simple 3    playing  Simple 1
Simple 3    playing  Simple 2
Simple 3    playing  Simple 3
Simple 3    playing  Optimal   
Optimal     playing  Random    
Optimal     playing  Simple 1
Optimal     playing  Simple 2
Optimal     playing  Simple 3
Optimal     playing  Optimal   

Tournament Results -- winnings for Strategy A
                        Strategy B
Strategy A    | Random     | Simple 1 | Simple 2 | Simple 3 | Optimal    | Total
_______________________________________________
Random        |2532

In [347]:
""" First cut learning.
    Will use logistic regression to set each of the vector parameters individually.
    
    Inline training -- play one game, instantly update.
    
    Train only A player.
    
    """
import math
def logit(x):
    p = 1 / ( 1 + math.exp(-x))
    return p

def logodds(p):
    if p == 0: return -10
    elif p == 1: return 10
    else:
        x = math.log(p / (1-p))
        return x


def train_A_logit(num_games = 10 ** 6,num_updates = 20 , alpha = 0.005, decksize = 6,
                  start_strategy =[], strategyB =[],verbose = False,learnvsaverage = False,diag = False):

    import random
    import copy
    
    # set paramaters and counters

    d = Deck(decksize)
    
    if start_strategy == [] :
        paramvector = [0 for i in range(decksize)]  # starts random parameter = 0 -> prob 50%
    else:
        paramvector = [logodds(p) for p in start_strategy]
        
    

    if strategyB == []:   # default strategy B is fold for lower half, call for larger half
        strategyB = [i > decksize / 2.0 for i in range(decksize)]
    sb = vectorstrat(strategyB)
    winnings = 0
    winnings_temp = 0
    training_updates =[]
    games_update = num_games / num_updates
    
    
    for i in range(num_games+1):
        
        # set strategy to current logit of parameters
        strategyvector = [logit(x) for x in paramvector]
        sa = vectorstrat(strategyvector)

        #play a game
        result = playgame(d,sa,sb,verbose = False)
      
        
        # direction of update is positive for a plus action.  Maybe change action from string to +/- 1
        if result['playA'] == "Raise":
            direction = 1
        else:
            direction = -1
        
        # update strategy.  Alpha is change rate parameter.  
        # Result * direction positive when aggression paid off or passive resulted in loss -> increase prob.
        # idea -- change result to result - average
        avgwins = 0
        if learnvsaverage:
            avgwins = winnings / i
            
        paramvector[result['cardA']] += alpha * (result['winA']-avgwins) *  direction
        
      
        # avoid over/underflow errors  Likely a better method, but this works within 1%
        paramvector[result['cardA']] = max(-10,paramvector[result['cardA']])
        paramvector[result['cardA']] = min(10,paramvector[result['cardA']])
        
    
        
        # track performance
        winnings += result['winA']
        winnings_temp += result['winA']
 
        # periodically update
        if i % games_update == 0 :
            if verbose:
                if i > 0: 
                    tempwinrate = round(winnings_temp / games_update,4) 
                else: 
                    tempwinrate = "       "
                print(i, tempwinrate,[round(logit(x)*100,2) for x in paramvector])    
            training_updates += [{'games':i,'tempwins':winnings_temp,'logodds':copy.deepcopy(paramvector)}]
            winnings_temp = 0
    
    if verbose:
        print("Done")
        winrate = winnings / num_games
        print("Winrate = ", winrate)
        print([round(x,4) for x in paramvector])
        strategyvector = [round(logit(x)*100,2)  for x in paramvector]
        print(strategyvector)
        
    return {'num_games':num_games, 'winnings':winnings,'final model':paramvector,'training_updates':training_updates}
        
    



Now lets set hyperparameters.  Key are alpha -- the adjustment rate and number of games.  Tests are performance (win rate, should be high) and stability (should be low)

In [348]:
"""Compare learning.  
   Compares two learning set-ups and graphs covergence stats.  
   Use to set hyperparameters (num_games, alpha) and test covergence
   Moving from ad-hoc review of performance to something better.

    TO DO (maybe)
    make graphics

"""
def av_vol(paramseries):
    # computes the moving GARCH vols of a timeseries of vectors, then takes the mean of the vector at each time.
    
    decayfactor = .5
    
    movingvars = [0 for i in paramseries[0]]
    previous = paramseries[0]
    av_vols =[]
    
    for newvalues in paramseries:
        movingvars = [decayfactor * oldmoving + (1 - decayfactor) * (old-new)**2 
                      for oldmoving,old,new in zip(movingvars,previous,newvalues)]
        movingvols = [x**0.5 for x in movingvars]
        av_vols += [(sum(movingvols)/len(movingvols))]
        previous = newvalues
        
    return av_vols
        
    


def compare (args,num_games = 10 ** 6, num_updates = 20):
    
    # args array of - dict.  'name' 'decksize' 'alpha' 'initial' 'opponent'
    
    tempwins = []
    games = []
    logodds = []
    performance = []
    stability = []
    
    for arg in args:
    
        arg['tempwins'] = []
        arg['games']= []
        arg['logodds'] = []
        arg['performance'] = []
        arg['stability'] = []
        
        results = train_A_logit(num_games, num_updates, 
                             alpha = arg['alpha'], decksize = arg['decksize'],
                             start_strategy = arg['initial'], strategyB = arg['opponent'],verbose = False)
  
        # transform results so they can be accessed easily -- Find better way to pass these.
    
        arg['tempwins'] += [[update['tempwins'] for update in results['training_updates']]]
        arg['games'] += [[update['games'] for update in results['training_updates']]]
        arg['logodds'] += [[update['logodds'] for update in results['training_updates']]]

        # compute performance and stability
        arg['performance'] += [ (win / (num_games/num_updates)) for win in arg['tempwins'][-1]]
        arg['stability'] += [av_vol(arg['logodds'][-1])]
 
    
    
    print ("                   ",end ="")
    for arg in args: 
        print(arg['name'],"               ",end = "")
    print("")
    print ("Games      ",end ="")
    for arg in args: 
        print("Performance   Stability   ",end ="")
    print("")
    
    #below is very ugly but cant figure out a better way
    if len(args) == 2:
        for g,p1,s1,p2,s2 in zip(args[0]['games'][0],args[0]['performance'],args[0]['stability'][0],
                                 args[1]['performance'],args[1]['stability'][0]):
            print(g,"     ",round(p1,4),"      ",round(s1,4),"     ",round(p2,4),"      ",round(s2,4))
    elif len(args)== 3:
        for g,p1,s1,p2,s2,p3,s3 in zip(args[0]['games'][0],args[0]['performance'],args[0]['stability'][0],
                                 args[1]['performance'],args[1]['stability'][0],
                                 args[2]['performance'],args[2]['stability'][0]):
            print(g,"     ",round(p1,4),"      ",round(s1,4),"     ",round(p2,4),"      ",round(s2,4),
                 "     ",round(p3,4),"      ",round(s3,4))
    else:
        for g,p1,s1,p2,s2,p3,s3,p4,s4 in zip(args[0]['games'][0],args[0]['performance'],args[0]['stability'][0],
                                 args[1]['performance'],args[1]['stability'][0],
                                 args[2]['performance'],args[2]['stability'][0],
                                 args[3]['performance'],args[3]['stability'][0],):
            print(g,"     ",round(p1,4),"      ",round(s1,4),"     ",round(p2,4),"      ",round(s2,4),
                 "     ",round(p3,4),"      ",round(s3,4), "     ",round(p4,4),"      ",round(s4,4))


In [349]:
firstargs = {'name':"Alpha 0.5", 'decksize':6, 'alpha': 0.5,'initial':[],'opponent':[]}
secondargs = {'name':"Alpha 0.05", 'decksize':6, 'alpha':0.05,'initial':[],'opponent':[]}
thirdargs = {'name':"Alpha 0.005", 'decksize':6, 'alpha':0.005,'initial':[],'opponent':[]}
fourthargs = {'name':"Alpha 0.0005", 'decksize':6, 'alpha':0.0005,'initial':[],'opponent':[]}
args = [firstargs,secondargs,thirdargs,fourthargs]
c = compare(args, num_games = 10**6,num_updates = 100)

                   Alpha 0.5                Alpha 0.05                Alpha 0.005                Alpha 0.0005                
Games      Performance   Stability   Performance   Stability   Performance   Stability   Performance   Stability   
0       0.0001        0.0       0.0001        0.0       0.0001        0.0       -0.0002        0.0
10000       0.241        4.2426       0.2834        5.7099       0.1865        1.4396       0.1073        0.0912
20000       0.2332        4.8187       0.2789        4.3045       0.2276        1.7237       0.1351        0.1176
30000       0.226        4.4115       0.2729        3.4719       0.2516        1.5623       0.1525        0.1341
40000       0.2078        3.9384       0.2441        2.8464       0.2636        1.2223       0.1572        0.1452
50000       0.2297        3.7285       0.261        2.2138       0.2713        0.9846       0.1713        0.1633
60000       0.2296        3.0741       0.2766        1.6732       0.2786        0.8364      

Above is analysis of hyperparamer setting for alpha and num_games.

Conclusions:  stability is directly related to alpha, and rapidly improves.  
0.5 is obviously far too high -- stability after 1MM is about the same as others at the begining.  
0.05 also has high instability for quite a while
0.005 looks better with both metrics seeming to hit a plateau at about 200K games
0.0005 covernes more slowly, getting to (more stable, roughly == in profit) plateau at 750K

Going forward, use alpha = 0.005, at least 200K games for a 6 card deck.
Next experiment -- effect of deck size.  Should be slower.

In [304]:
firstargs = {'name':"Deck 6", 'decksize':6, 'alpha': 0.005,'initial':[],'opponent':[]}
secondargs = {'name':"Deck 13", 'decksize':13, 'alpha':0.005,'initial':[],'opponent':[]}
thirdargs = {'name':"Deck 52", 'decksize':52, 'alpha':0.005,'initial':[],'opponent':[]}
args = [firstargs,secondargs,thirdargs]
c = compare(args, num_games = 10**6,num_updates = 20)

                   Deck 6                Deck 13                Deck 52                
Games      Performance   Stability   Performance   Stability   Performance   Stability   
0       -0.0        0.0       -0.0        0.0       -0.0001        0.0
50000       0.0252        3.7506       0.036        3.2821       -0.0863        1.7719
100000       0.044        2.6858       0.1033        2.5051       0.0658        1.6247
150000       0.0468        1.9239       0.0975        1.8002       0.0877        1.3154
200000       0.0533        1.4084       0.088        1.2892       0.1149        1.0758
250000       0.0428        1.0423       0.0896        0.9433       0.0797        0.8663
300000       0.0467        0.7445       0.0973        0.6958       0.1038        0.7029
350000       0.0575        0.5323       0.0825        0.5167       0.1045        0.5621
400000       0.0532        0.3942       0.1098        0.3935       0.1125        0.4516
450000       0.0541        0.3472       0.0892    

6 Deck was similar to above.  Hits performance plateau aroiund 350K, stabiliyt at 650K
13 Deck a bit longer -- performance flat by 400K, stability lower and plateu ing at 750K
52 Deck performance was more volatile.  flattened at 600K, stable at 800K.

It is likely that performance becomes stable before parameters becasue there is a large plateau where true expected performance is close to flat.  ML finds this area, but then has trouble finding absolute max.

Next Experiment.  Learn based on return - average -- i.e. excess return vs current expectation.  Code of learn changed to allow this.

In [309]:
firstargs = {'name':"Deck 6", 'decksize':6, 'alpha': 0.005,'initial':[],'opponent':[]}
secondargs = {'name':"Deck 6", 'decksize':6, 'alpha': 0.005,'initial':[],'opponent':[],'learnvsaverage':True}
args = [firstargs,secondargs]
c = compare(args, num_games = 10**6,num_updates = 20)

                   Deck 6                Deck 6                
Games      Performance   Stability   Performance   Stability   
0       0.0        0.0       -0.0        0.0
50000       0.0407        3.661       0.0276        3.6734
100000       0.0498        2.6148       0.0449        2.6581
150000       0.0456        1.8727       0.0422        1.8977
200000       0.0492        1.3457       0.0561        1.3617
250000       0.0615        0.9683       0.0549        0.9777
300000       0.0642        0.7163       0.0362        0.7089
350000       0.042        0.5289       0.0462        0.5469
400000       0.0561        0.4406       0.0531        0.4456
450000       0.056        0.3309       0.0505        0.3985
500000       0.0588        0.2492       0.051        0.2934
550000       0.0592        0.1974       0.0438        0.274
600000       0.0329        0.1587       0.0419        0.2573
650000       0.0442        0.1201       0.0556        0.2137
700000       0.0698        0.1068       

Doesn't seem to matter much. I guess that given expected wins are ~0.05 = 0.10 and results are -2/-1/1/2, the effect is minor.


In [323]:
"""Second compare.  Used to compare outcomes.

   Moving from ad-hoc review of performance to something better.


'num_games':num_games, 'winnings':winnings,'final model':paramvector,'training_updates':training_updates

"""
def compare_outcomes (args):
    
    # args array of - dict.  'name' 'decksize' 'alpha' 'initial' 'opponent'
    
    results =[]
    
    print("Name  WinRate      Probvector")
    
    for arg in args:
            
        results = train_A_logit(num_games = arg['num_games'], num_updates = arg['num_updates'],  
                             alpha = arg['alpha'], decksize = arg['decksize'],
                             start_strategy = arg['initial'], strategyB = arg['opponent'],verbose = arg['verbose'])
        arg['winrate'] = results['winnings'] / results['num_games']
        arg['model_logodds'] = results['final model']
        arg['model_probs'] = [round(logit(x)*100,1) for x in results['final model']]
        
        print(arg['name']," $",round(arg['winrate'],2),"  ",arg['model_probs'])    
 
    
    
    

In [351]:
firstargs = {'name':"50%", 'decksize':6, 'alpha': 0.005,'initial':[],'opponent':[],'num_games':10**6,'num_updates':1,'verbose':False}
secondargs = {'name':"99%", 'decksize':6, 'alpha': 0.005,'initial':[.99,.99,.99,.99,.99,.99],'opponent':[],'num_games':10**6,'num_updates':1,'verbose':False}
thirdargs = {'name':"1%", 'decksize':6, 'alpha': 0.005,'initial':[.01,.01,.01,.01,.01,.01],'opponent':[],'num_games':10**6,'num_updates':1,'verbose':False}
args = [firstargs,secondargs,thirdargs]
compare_outcomes(args)

Name  WinRate      Probvector
50%  $ 0.27    [99.4, 99.9, 100.0, 0.0, 0.0, 100.0]
99%  $ 0.24    [99.9, 99.8, 100.0, 0.0, 100.0, 100.0]
1%  $ 0.24    [99.8, 99.6, 97.1, 0.0, 0.0, 0.0]


**INTERESTING**  When starting in the low prob corner, the algo never found that the best solution was to raise for a high card.

Also intersting that the solution against the default opponent [0,0,0,1,1,1] is to not raise on  4.  Will check this vs. optimal opponent  [0, 1/3,1/3,1,1,1], where the analytical solution is [2/3,0,0,1,1]

In [352]:
firstargs = {'name':"vs random", 'decksize':6, 'alpha': 0.005,'initial':[],'opponent':[.5,.5,.5,.5,.5,.5],'num_games':10**6,'num_updates':1,'verbose':False}
secondargs = {'name':"vs standard", 'decksize':6, 'alpha': 0.005,'initial':[],'opponent':[],'num_games':10**6,'num_updates':1,'verbose':False}
thirdargs = {'name':"vs optimal", 'decksize':6, 'alpha': 0.005,'initial':[],'opponent':[0,1.0/3.0,1.0/3.0,1,1,1],'num_games':10**6,'num_updates':1,'verbose':False}
args = [firstargs,secondargs,thirdargs]
compare_outcomes(args)

Name  WinRate      Probvector
vs random  $ 0.47    [73.0, 99.9, 100.0, 100.0, 100.0, 100.0]
vs standard  $ 0.27    [99.6, 99.9, 92.3, 0.0, 0.0, 100.0]
vs optimal  $ 0.06    [45.8, 42.7, 25.9, 0.0, 100.0, 100.0]


Next test --13 card learning from corners



In [353]:
firstargs = {'name':"50%", 'decksize':13, 'alpha': 0.005,'initial':[],'opponent':[],'num_games':10**6,'num_updates':1,'verbose':False}
secondargs = {'name':"99%", 'decksize':13, 'alpha': 0.005,'initial':[.99,.99,.99,.99,.99,.99,.99,.99,.99,.99,.99,.99,.99],'opponent':[],'num_games':10**6,'num_updates':1,'verbose':False}
thirdargs = {'name':"1%", 'decksize':13, 'alpha': 0.005,'initial':[.01,.01,.01,.01,.01,.01,.01,.01,.01,.01,.01,.01,.01],'opponent':[],'num_games':10**6,'num_updates':1,'verbose':False}
args = [firstargs,secondargs,thirdargs]
compare_outcomes(args)

Name  WinRate      Probvector
50%  $ 0.11    [75.0, 65.0, 59.6, 56.3, 50.9, 26.0, 0.2, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0]
99%  $ 0.08    [67.9, 64.1, 62.8, 51.5, 47.6, 27.1, 0.3, 0.0, 100.0, 100.0, 100.0, 100.0, 100.0]
1%  $ 0.05    [70.6, 64.6, 58.9, 52.7, 42.9, 29.2, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Seems odd that the 1% staring point never tried raising on a high card -- if it did, that should have been reinforced. 

Let's try longer run time, to see if it eventually finds the right thing.  Back to 6 cards to get faster convergence.  Need to adjust compare_outcomes to take numgames as a scenario level argument

In [355]:
firstargs = {'name':"1M", 'decksize':6, 'num_games':10**6,'num_updates':1,'verbose':False,'alpha': 0.005,'initial':[.01,.01,.01,.01,.01,.01],'opponent':[]}
secondargs = {'name':"100M", 'decksize':6, 'num_games':10**8,'num_updates':20,'verbose':True,'alpha': 0.005,'initial':[.01,.01,.01,.01,.01,.01],'opponent':[]}

args = [firstargs,secondargs]
compare_outcomes(args)

Name  WinRate      Probvector
1M  $ 0.24    [99.9, 99.7, 100.0, 0.0, 0.0, 0.0]
0         [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
5000000 0.2469 [99.2, 99.99, 99.79, 0.0, 0.0, 0.0]
10000000 0.2506 [99.95, 99.99, 99.98, 0.0, 0.0, 0.0]
15000000 0.2496 [99.99, 99.97, 99.79, 0.0, 0.0, 0.0]
20000000 0.2508 [99.98, 99.95, 99.98, 0.0, 0.0, 0.0]
25000000 0.2483 [99.05, 100.0, 99.17, 0.0, 0.0, 0.0]
30000000 0.2496 [99.99, 99.99, 99.88, 0.0, 0.0, 0.0]
35000000 0.249 [99.96, 99.58, 97.17, 0.0, 0.0, 0.0]
40000000 0.2506 [99.98, 99.99, 99.99, 0.0, 0.0, 0.0]
45000000 0.2496 [99.96, 99.53, 99.99, 0.0, 0.0, 0.0]
50000000 0.248 [99.98, 99.91, 96.3, 0.0, 0.0, 0.0]
55000000 0.2496 [99.82, 99.99, 99.84, 0.0, 0.0, 0.0]
60000000 0.2498 [99.96, 99.91, 99.96, 0.0, 0.0, 0.0]
65000000 0.248 [99.85, 99.85, 99.97, 0.0, 0.0, 0.0]
70000000 0.2492 [99.91, 99.99, 99.05, 0.0, 0.0, 0.0]
75000000 0.2488 [99.93, 99.9, 99.89, 0.0, 0.0, 0.0]
80000000 0.2496 [99.96, 99.91, 99.99, 0.0, 0.0, 0.0]
85000000 0.2495 [99.81, 99.95, 99.85, 0

Interesting.  Didn't find the 'obvious' strategy to raise on a high card.  Not only that, but the prob went *down* -- Started 1% which is log odds of around 2. End result is logodds at the -10 cap.  